# Clearing address from dataframe origin - Tratando base de dados dos endereços do dataframe original. 

In [ ]:
!pip install geopandas  

In [ ]:
!pip install shapely

In [3]:
import matplotlib 
import pandas as pd
import geopandas as gpd

In [5]:
from shapely.geometry import Point, Polygon  

In [6]:
from geopandas import geoseries 

- csv file generated from building.ipynb notebook. -- Arquivo csv gerado a partir do notebook building.ipynb. 

In [85]:
ppi_address_ajust = '/content/address_ppi.csv'
ppi_address_ajust = pd.read_csv(ppi_address_ajust, encoding='ISO-8859-1')
ppi_address_ajust.head()

,Unnamed: 0,Empreendimento,UF
0,0,Aeroporto de Salvador/ BA,BA
1,1,Aeroporto de Florianópolis/ SC,SC
2,2,Aeroporto de Fortaleza/ CE,CE
3,3,Aeroporto de Porto Alegre/ RS,RS
4,4,Aeroporto de Maceió/ AL,AL


In [ ]:
ppi_address_ajust.drop('Unnamed: 0', axis = 1)

In [87]:
ppi_address_ajust.dtypes

Unnamed: 0         int64
Empreendimento    object
UF                object
dtype: object

In [ ]:
ppi_address_ajust.values

In [89]:
ppi_address_ajust['ppi_ajust_cols'] = ppi_address_ajust['Empreendimento'] + '/' + ppi_address_ajust['UF']

In [ ]:
ppi_address_ajust

In [91]:
pd.set_option('max_rows', 200)
pd.set_option('max_columns', 100)

- 'brazil_states' 

# Tools Geocode, for point location 
 - return GeometryCollection empty

In [ ]:
gpd.tools.geocode(ppi_address_ajust['Empreendimento'], provider = 'nominatim', user_agent='UF Builds', country_bias = 'Brazil')

In [ ]:
ppi_address_ajust['Geom'] = gpd.tools.geocode(ppi_address_ajust['Empreendimento'], provider = 'nominatim', user_agent='UF Builds', country_bias = 'brazil_states')['geometry']

In [ ]:
ppi_address_ajust

In [ ]:
ppi_address_ajust.loc[ppi_address_ajust['Geom'] == 'GEOMETRYCOLLECTION EMPTY', 'Geom'].head()

In [ ]:
ppi_address_ajust.drop_duplicates() # if have, < inplace = True > 

# Geocoders implementation for locations acess - implementação de acesso das localizações usando o Geocoders 
- return None

In [92]:
import geopy.geocoders

In [93]:
from geopy.geocoders import Nominatim

In [94]:
geopy.geocoders.options.default_user_agent = 'my_app/1'

In [95]:
geopy.geocoders.options.default_timeout = 7

In [96]:
geolocator = Nominatim()

In [97]:
print(geolocator.headers)

{'User-Agent': 'my_app/1'}


In [98]:
print(geolocator.timeout)

7


- Test aplication 

In [99]:
# locator = Nominatim(user_agent='myGeocoder')
location = geolocator.geocode('Aeroporto de Salvador/ BA')

In [100]:
print(location)

Aeroporto, Travessa Santos Dumont, Lauro de Freitas, Região Geográfica Imediata de Salvador, Região Metropolitana de Salvador, Região Geográfica Intermediária de Salvador, Bahia, Região Nordeste, 42700000, Brasil


In [101]:
print(location.point)

12 54m 15.1844s S, 38 20m 27.9298s W


In [102]:
location.point

Point(-12.904217899999999, -38.341091608139536, 0.0)

In [103]:
from geopy.extra.rate_limiter import RateLimiter

- Search all address from dataframe

In [104]:
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [105]:
ppi_address_ajust['ppi_geocode'] = ppi_address_ajust['Empreendimento'].apply(geocode)

In [ ]:
ppi_address_ajust

- plot points of address 

In [107]:
ppi_address_ajust['point'] = ppi_address_ajust['ppi_geocode'].apply(lambda loc: tuple(loc.point) if loc else None)

- ajust None's

In [108]:
pd.set_option('max_rows', 200)
pd.set_option('max_columns', 100)

In [ ]:
ppi_address_ajust

In [ ]:
ppi_address_ajust.isna()

In [ ]:
# ppi_point = ppi_address_ajust.drop(['Unnamed: 0',	'Empreendimento',	'UF',	'ppi_ajust_cols'], axis=1)

- New position from point values not null. 

In [139]:
ppi_address_ajust.sort_values(by='point', ignore_index=True)

,ppi_geocode,point
0,"(Aeroporto, Acesso a sede Trensurb, Humaitá, P...","(-29.9877277, -51.18287, 0.0)"
1,"(Rodovia Acesso ao Aeroporto, Carianos, Ribeir...","(-27.6819223, -48.5303734, 0.0)"
2,"(Canal Adutor 1 PCH Pery, Curitibanos, Região ...","(-27.4563247, -50.6420465, 0.0)"
3,"(Atendimento ao Turista, BR-101, Rua XV de Nov...","(-26.2965993, -48.88310175, 0.0)"
4,(Aeroporto Internacional Governador André Fran...,"(-23.43779495, -46.481320528800595, 0.0)"
5,(RIOgaleão - Aeroporto Internacional Tom Jobim...,"(-22.81122025, -43.258620204081936, 0.0)"
6,"(Centro de Atendimento ao Usuário, km 133,5, R...","(-22.647567, -43.1870608, 0.0)"
7,"(Aeroporto de Macaé, Rua Hildebrando Alves Bar...","(-22.343171050000002, -41.76577364545359, 0.0)"
8,"(Ponte sobre o Rio Paraibuna, Sítio São José, ...","(-22.0112792, -43.266435, 0.0)"
9,(Aeroporto Internacional de Vitória - Eurico d...,"(-20.256092950000003, -40.28135718192593, 0.0)"


In [138]:
ppi_address_ajust.index

RangeIndex(start=0, stop=191, step=1)

# To remember codes: 

In [ ]:
#show
# ppi_address_ajust.index.values
# ppi_address_ajust.Geom.values
# type(ppi_address_ajust.Geom.values)

# rename 
# ppi_address_ajust.rename(columns= {'Unnamed: 0': 'Rows_place'}, inplace=True )